In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('F1-2023.db')
c = conn.cursor()

In [14]:
def score_and_matrix(gp_id):
    player_names = ["Wonde","Tome", "Hrici", "Kajlo", "Sasu", "OG"]
    matrix_of_ans = []
    score = []
    for i in range(len(player_names)):
        matrix_i = [None for _ in range(15)]
        #correct answers
        c.execute(f'''SELECT a.question_id FROM bets a JOIN ground_truth b
                     ON a.grand_prix_id = b.grand_prix_id
                     AND a.answer = b.correct_ans
                     AND a.question_id = b.question_id
                     WHERE a.player_id = {i+1}
                     AND a.grand_prix_id = {gp_id}''')
        myresult = c.fetchall()
        correct_pts = len(myresult)*2
        minus_pts = 0

        #matrix
        for q_id in myresult:
            matrix_i[q_id[0]-1] = True

        #if DNF player was correct
        if (6,) in myresult:
            matrix_i[5] = True
        else:
            minus_pts += 1
            matrix_i[5] = False

        #incorrect answers
        c.execute(f'''SELECT a.question_id FROM bets a JOIN ground_truth b
                 ON a.grand_prix_id = b.grand_prix_id
                 AND a.answer <> b.correct_ans
                 AND a.question_id = b.question_id
                 WHERE a.player_id = {i+1}
                 AND a.grand_prix_id = {gp_id}''')

        myresult1 = c.fetchall()
        for ans in myresult1:
            if ans[0] >= 1 and ans[0] <= 11 and ans[0] != 6:
                minus_pts += 1
            if ans[0] != 6:
                matrix_i[ans[0]-1] = False

        matrix_of_ans.append(matrix_i)
        score.append(max(correct_pts - minus_pts, -6))
    return(score, matrix_of_ans)
print(score_and_matrix(1))

([15, 18, 18, 8, 18, -6], [[True, True, False, False, False, False, True, False, True, True, True, True, True, True, True], [True, True, False, False, False, False, True, True, True, True, True, True, True, True, True], [True, True, True, False, False, False, True, False, True, True, True, True, True, True, True], [True, False, False, False, False, False, True, False, True, True, True, True, False, True, False], [True, True, True, False, False, False, True, False, True, True, True, True, True, True, True], [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]])


In [20]:
jedah = ''
player_names = ["Wonde","Tome", "Hrici", "Kajlo", "Sasu", "OG"]
def answers(gp_id):
    d = {"Wonde":[None for _ in range(15)],"Tome":[None for _ in range(15)], "Hrici":[None for _ in range(15)], "Kajlo":[None for _ in range(15)], "Sasu":[None for _ in range(15)], "OG":[None for _ in range(15)]}
    c.execute(f'''SELECT * FROM bets
                  WHERE grand_prix_id = {gp_id}''')

    myresult = c.fetchall()
    for ans in myresult:
        d[player_names[int(ans[1])-1]][ans[0]-1] = ans[3]

    df = pd.DataFrame(data=d)
    df_bool = pd.DataFrame(score_and_matrix(1)[1]).T
    def color_boolean(val):
      color =''
      if df_bool == False:
        color = 'red'
      elif df_bool == True:
        color = 'black'
      return f'color: {color}'
    df.style.apply(lambda c: df_bool[c.name].apply(color_boolean))
    print(df)

answers(2)

      Wonde     Tome    Hrici        Kajlo         Sasu           OG
0       PER      PER      PER          ALO          PER          PER
1       ALO      ALO      ALO          PER          LEC          ALO
2       VER      SAI      VER          VER          ALO          VER
3       VER      VER      VER          VER          VER          PER
4       DEV      DEV      SAR          DEV          SAR          SAR
5       SER      SER      NOR          SAR          TSU          STR
6       Ano      Ano      Ano          Ano          Nie          Nie
7       Ano      Ano      Ano          Ano          Ano          Ano
8       Nie      Ano      Ano          Ano          Ano          Ano
9       Nie      Ano      Nie          Nie          Nie          Nie
10      Ano      Ano      Ano          Ano          Ano          Ano
11      Nie      Nie      Nie          Nie          Nie          Nie
12      Nie      Nie      Nie  Ano-Redbull  Ano-Redbull  Ano-Redbull
13      PER      PER      PER     

In [5]:
conn.close()